<a href="https://colab.research.google.com/github/meetarora10/veersa-hack/blob/main/model_chatbot_use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Create an offload folder if not exists
os.makedirs("offload", exist_ok=True)

def load_model():
    model_name = "TheBloke/vicuna-7b-1.1-HF"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        offload_folder="offload"
    )
    return tokenizer, model

def generate_response(tokenizer, model, chat_history, user_input):
    # Add the latest user input to chat history
    chat_history += f"\nUser: {user_input}\nAssistant:"

    # Tokenize and generate response
    inputs = tokenizer(chat_history, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the new assistant reply (after last "Assistant:")
    assistant_reply = response.split("Assistant:")[-1].strip()

    # Append to chat history for memory
    chat_history += f" {assistant_reply}"
    return assistant_reply, chat_history

def main():
    tokenizer, model = load_model()
    print("Chatbot ready. Type your question (or 'exit' to quit).")

    chat_history = "You are a helpful telehealth assistant. Answer carefully."

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            break
        reply, chat_history = generate_response(tokenizer, model, chat_history, user_input)
        print("Assistant:", reply)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:609: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Chatbot ready. Type your question (or 'exit' to quit).

You: I am having fever from last 2 days
Assistant: I'm sorry to hear that. Can you tell me more about your symptoms? Do you have a fever that is getting worse or better? Do you have any other symptoms such as body aches, headache, or cough? Have you been to a doctor or healthcare provider yet?

You: it is not getting better and i am having cough and body ache also
Assistant: It sounds like you may have a viral infection, such as the flu or a cold. It is important to see a doctor or healthcare provider as soon as possible to get a proper diagnosis and treatment. In the meantime, there are some things you can do at home to help relieve your symptoms.

1. Drink plenty of fluids to stay hydrated.
2. Take acetaminophen or ibuprofen to help reduce fever and relieve body aches.
3. Rest and get plenty of sleep.
4. Use a humidifier to help relieve cough and congestion.
5. Avoid alcohol, caffeine,

You: okay i will do so
Assistant: Great! R